<a href="https://colab.research.google.com/github/Spili11/DS_Praktikum/blob/main/assignment_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from google.colab import drive

from tensorflow.keras.preprocessing import image_dataset_from_directory


In [2]:
drive.mount('/content/drive')

NameError: ignored

In [26]:
import os
import shutil
import random

In [22]:
mypath = "/content/drive/My Drive/DS_Praktikum/assignment05_data/"
a = ['train', 'validation']
b = ['batman', 'superman', 'spiderman']
for x, y in [(x,y) for x in a for y in b]:
  if not os.path.isdir(mypath + x + "/"+ y):
    os.makedirs(mypath + x + "/" + y)

In [132]:
validation_ratio = 0.2

batman_raw = os.listdir("/content/drive/My Drive/DS_Praktikum/assignment05_data/batman")
batman_validation = random.sample(batman_raw, k = int((len(batman_raw) * validation_ratio) // 1))
batman_train = list(set(batman_raw) - set(batman_validation))

superman_raw = os.listdir("/content/drive/My Drive/DS_Praktikum/assignment05_data/superman")
superman_validation = random.sample(superman_raw, k = int((len(superman_raw) * validation_ratio) // 1))
superman_train = list(set(superman_raw) - set(superman_validation))

spiderman_raw = os.listdir("/content/drive/My Drive/DS_Praktikum/assignment05_data/spiderman")
spiderman_validation = random.sample(spiderman_raw, k = int((len(spiderman_raw) * validation_ratio) // 1))
spiderman_train = list(set(spiderman_raw) - set(spiderman_validation))


In [133]:
print(len(batman_raw), len(batman_validation), len(batman_train))
print(len(spiderman_raw), len(spiderman_validation), len(spiderman_train))
print(len(superman_raw), len(superman_validation), len(superman_train))

292 58 234
290 58 232
292 58 234


In [137]:
for x in batman_train:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/batman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/train/batman/" + x
  
  shutil.copyfile(original, target)

In [138]:
for x in batman_validation:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/batman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/validation/batman/" + x
  
  shutil.copyfile(original, target)

In [139]:
for x in superman_train:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/superman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/train/superman/" + x
  
  shutil.copyfile(original, target)

In [140]:
for x in superman_validation:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/superman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/validation/superman/" + x
  
  shutil.copyfile(original, target)

In [141]:
for x in spiderman_train:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/spiderman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/train/spiderman/" + x
  
  shutil.copyfile(original, target)

In [142]:
for x in spiderman_validation:
  original = "/content/drive/My Drive/DS_Praktikum/assignment05_data/spiderman/" + x
  target = "/content/drive/My Drive/DS_Praktikum/assignment05_data/validation/spiderman/" + x
  
  shutil.copyfile(original, target)